In [1]:
# Code from: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/


from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [2]:
## Code from: https://stackabuse.com/accessing-the-twitter-api-with-python/#disqus_thread

# Import the Twython class
from twython import Twython
import json

import pandas as pd

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Number of users
Number_of_users = 50

In [37]:
## QUERIES!
## Create our queries

query1 = {'q': 'politics',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query2 = {'q': 'economics',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query3 = {'q': 'technology',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query4 = {'q': 'culture',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query5 = {'q': 'entertainment',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query6 = {'q': 'nature',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query7 = {'q': 'science',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query8 = {'q': 'climate',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query9 = {'q': 'sports',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query10 = {'q': 'business',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query11 = {'q': 'health',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query12 = {'q': 'arts',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

In [38]:
# Search tweets
def search(query):
    dict_ = {'user': [], 'date': [], 'hashtags': [], 'full_text': [], 'favorite_count': []}
    for status in python_tweets.search(**query)['statuses']:
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
        dict_['full_text'].append(status['full_text'])
        dict_['favorite_count'].append(status['favorite_count'])
    return dict_

# Note that these are the most popular tweets containing the word "..." in the past 7 days. 
# To explore data back in history, you'll need to purchase the Premium or Enterprise plan of the Search API.

In [39]:
df1 = pd.DataFrame(search(query1))
df2 = pd.DataFrame(search(query2))
df3 = pd.DataFrame(search(query3))
df4 = pd.DataFrame(search(query4))
df5 = pd.DataFrame(search(query5))
df6 = pd.DataFrame(search(query6))
df7 = pd.DataFrame(search(query7))
df8 = pd.DataFrame(search(query8))
df9 = pd.DataFrame(search(query9))
df10 = pd.DataFrame(search(query10))
df11 = pd.DataFrame(search(query11))
df12 = pd.DataFrame(search(query12))
## Number of possible combinations of 5 out of 10:
## 10x9x8x7x6 = 30240
## 5x4x3x2x1 = 120
## 30240/120 = 252 combinations

In [40]:
df1['label'] = 'politics'
df2['label'] = 'economics'
df3['label'] = 'technology'
df4['label'] = 'culture'
df5['label'] = 'entertainment'
df6['label'] = 'nature'
df7['label'] = 'science'
df8['label'] = 'climate'
df9['label'] = 'sports'
df10['label'] = 'business'
df11['label'] = 'health'
df12['label'] = 'arts'

In [41]:
df1

,user,date,hashtags,full_text,favorite_count,label
0,TrulyMonica,Tue May 18 06:57:35 +0000 2021,[],“Block Covid resources such as beds and releas...,5018,politics
1,Sanju_Verma_,Tue May 18 06:19:50 +0000 2021,"[COVID19, CongressToolkitExposed]",ToolKit by @INCIndia is another embarrassing t...,1591,politics
2,ddale8,Mon May 17 22:25:59 +0000 2021,[],Crenshaw claimed the GOP brief just asked the ...,7128,politics
3,Barzini_Baaz,Tue May 18 13:55:28 +0000 2021,[کورونا_پھیلاتی_مریم],RT @Ch_danishh: #کورونا_پھیلاتی_مریم\n\nTo all...,0,politics
4,SanataniSeeker,Tue May 18 13:55:27 +0000 2021,[],"RT @amishra77: The truth is, no one needs a to...",0,politics
...,...,...,...,...,...,...
95,drfarmertx,Tue May 18 13:55:16 +0000 2021,[],"RT @WHCOS: ""There’s a blue-collar sensibility ...",0,politics
96,MetalHead_1969,Tue May 18 13:55:16 +0000 2021,[],@newsmax Nothing is a coincidence in politics.,0,politics
97,hankt72,Tue May 18 13:55:15 +0000 2021,[],"RT @Telegraph: ❌ ""In view of his ludicrous rem...",0,politics
98,judypiejones,Tue May 18 13:55:15 +0000 2021,[],RT @davidhogg111: Powerful women are taking th...,0,politics


In [44]:
total_train_DF = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12])
total_train_DF

,user,date,hashtags,full_text,favorite_count,label
0,TrulyMonica,Tue May 18 06:57:35 +0000 2021,[],“Block Covid resources such as beds and releas...,5018,politics
1,Sanju_Verma_,Tue May 18 06:19:50 +0000 2021,"[COVID19, CongressToolkitExposed]",ToolKit by @INCIndia is another embarrassing t...,1591,politics
2,ddale8,Mon May 17 22:25:59 +0000 2021,[],Crenshaw claimed the GOP brief just asked the ...,7128,politics
3,Barzini_Baaz,Tue May 18 13:55:28 +0000 2021,[کورونا_پھیلاتی_مریم],RT @Ch_danishh: #کورونا_پھیلاتی_مریم\n\nTo all...,0,politics
4,SanataniSeeker,Tue May 18 13:55:27 +0000 2021,[],"RT @amishra77: The truth is, no one needs a to...",0,politics
...,...,...,...,...,...,...
95,10x10Photobooks,Tue May 18 13:53:58 +0000 2021,[],RT @PhotoLondonFair: The inaugural \n@BRSphoto...,0,arts
96,testayzay,Tue May 18 13:53:55 +0000 2021,[],"RT @mygbebe: Something about ""That debate is v...",0,arts
97,anELFforSJ,Tue May 18 13:53:53 +0000 2021,[],"RT @mygbebe: Something about ""That debate is v...",0,arts
98,Agustdeee1,Tue May 18 13:53:52 +0000 2021,[],"RT @mygbebe: Something about ""That debate is v...",0,arts


In [45]:
# import itertools 

# numbers = [1,2,3,4,5,6,7,8,9,10,11,12]

# combinations = list(itertools.combinations(numbers, 5)) 

# df_combinations = []

# for i in range(Number_of_users):

#     df_combinations.append(['df'+str(combinations[i][0]),
#                             'df'+str(combinations[i][1]),
#                             'df'+str(combinations[i][2]),
#                             'df'+str(combinations[i][3]),
#                             'df'+str(combinations[i][4])])
#     name = 'user'+str(i)
#     globals()[name] = pd.concat([globals()[df_combinations[i][0]],
#                                  globals()[df_combinations[i][1]],
#                                  globals()[df_combinations[i][2]],
#                                  globals()[df_combinations[i][3]],
#                                  globals()[df_combinations[i][4]]])

# print('number of user:', len(df_combinations))

In [ ]:
# # load the dataset
# data = open('data/corpus', encoding="utf8").read()
# labels, texts = [], []
# for i, line in enumerate(data.split("\n")):
#     content = line.split()
#     labels.append(content[0])
#     texts.append(" ".join(content[1:]))

# # create a dataframe using texts and lables
# trainDF = pandas.DataFrame()
# trainDF['text'] = texts
# trainDF['label'] = labels

In [194]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(total_train_DF['full_text'], total_train_DF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [195]:
encoder.classes_

array(['arts', 'business', 'climate', 'culture', 'economics',
       'entertainment', 'health', 'nature', 'politics', 'science',
       'sports', 'technology'], dtype=object)

In [48]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(total_train_DF['full_text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [49]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(total_train_DF['full_text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(total_train_DF['full_text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(total_train_DF['full_text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [50]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/glove.6B.300d.txt', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(total_train_DF['full_text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [51]:
total_train_DF['char_count'] = total_train_DF['full_text'].apply(len)
total_train_DF['word_count'] = total_train_DF['full_text'].apply(lambda x: len(x.split()))
total_train_DF['word_density'] = total_train_DF['char_count'] / (total_train_DF['word_count']+1)
total_train_DF['punctuation_count'] = total_train_DF['full_text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
total_train_DF['title_word_count'] = total_train_DF['full_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
total_train_DF['upper_case_word_count'] = total_train_DF['full_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [52]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

total_train_DF['noun_count'] = total_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'noun'))
total_train_DF['verb_count'] = total_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'verb'))
total_train_DF['adj_count'] = total_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'adj'))
total_train_DF['adv_count'] = total_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'adv'))
total_train_DF['pron_count'] = total_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [53]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [54]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

In [152]:
def prepare(df,df_part):
    tfidf_vect2 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
    tfidf_vect2.fit(df.stack())
    test_tfidf =  tfidf_vect2.transform(df_part)
    return test_tfidf


In [114]:
test = ['Could reporters stop asking if political leaders "believe" in climate change and start asking if they understand it instead', 
       'If I were ever abducted by aliens, the first thing I’d ask is whether they came from a planet where people also deny science.',
       'Alien: why should I not blow up this planet? Human: we are an advanced species A: how do you travel? H: we light old dinosaurs on fire']
df_test = pd.DataFrame(test)
df_test

,0
0,Could reporters stop asking if political leade...
1,"If I were ever abducted by aliens, the first t..."
2,Alien: why should I not blow up this planet? H...


In [201]:
tfidf_vect3 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect3.fit(test)
tfidf =  tfidf_vect3.transform(test)


In [105]:
import pickle

model = linear_model.LogisticRegression()
model.fit(xtrain_tfidf, train_y)
# save the model to disk
filename = 'AImodel.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [153]:
test_arr = prepare(df_test, df_test[0])

In [204]:
xvalid_tfidf

<300x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 5959 stored elements in Compressed Sparse Row format>

In [191]:
# test_arr = prepare(df_test)


result = loaded_model.predict(xvalid_tfidf)
print(result)

[ 7  2  6  6  6 11 10  3 10  6  7 10  8  0  2 10 11  9  6 10  8  7 11  6
  3 11  0  6  8  1  1  7  8  9  8  9  4  5  5  0  0 10  1  1  5  9 11  6
  3 10  2  4  1  4  7  0  1  8  3  2  3  9  4  2  7  9  8  3  3  8 11  1
  3  0  5  6  5 10 10  7  6  6  2  1 11 11  7  2  4  3  8  7  7  3  9  5
  6  1  9  5  3  8  1  8  6  8  9  0  6  4  2  2  2  2  4 11  0  7  8  5
  6  4  3  1 10 10  6  2  3  8 11  9  4  8  0  6  0  6 11  0  2  6  9  1
  8  8  1  9  8 11  1  1  7  2  6  8  0  5  7  4 11 11 11 11  6  2 10  0
  3  3  3  5  0  7  8  6  8 11  6  1  9  4  8  2 10  1  2  6  2  7  5  6
 11  5  2  8  8 11 10  5  4 10  8  5  6  5  6  3 10  7  9  8  5  0  6 11
 11  3  6  7  6  8  0  4  4  4  6  6  5  1  5  6  3  4 11  7  7 10  9 10
  0  2  7  2  4  8  6  2 11  3 10  6  8  4 11 10  5 10  6 11  6  8 11  4
  3  0  9  6  3  3  8  1 11  2  2  7  9  4 11  5  1  8  9  6  5 11  5  2
  4  0  2  5  2 10  6  7 10  7  5  4]


In [205]:
test_tfidf =  tfidf_vect.transform(test)

In [206]:
result3 = loaded_model.predict(test_tfidf)
print(result3)

[2 9 3]


In [197]:
topics = list(encoder.classes_)

In [199]:
result[0]

7

In [198]:
topics

['arts',
 'business',
 'climate',
 'culture',
 'economics',
 'entertainment',
 'health',
 'nature',
 'politics',
 'science',
 'sports',
 'technology']

In [207]:
result2 = []

for i in range(len(result3)):
    result2.append(topics[result3[i]]) 
    
result2

['climate', 'science', 'culture']

In [60]:
# # Naive Bayes on Count Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
# print ("NB, Count Vectors: ", accuracy)

# # Naive Bayes on Word Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("NB, WordLevel TF-IDF: ", accuracy)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("NB, N-Gram Vectors: ", accuracy)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("NB, CharLevel Vectors: ", accuracy)

In [65]:
# # Linear Classifier on Count Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
# print ("LR, Count Vectors: ", accuracy)

# # Linear Classifier on Word Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("LR, WordLevel TF-IDF: ", accuracy)

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("LR, N-Gram Vectors: ", accuracy)

# # Linear Classifier on Character Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("LR, CharLevel Vectors: ", accuracy)

In [63]:
# # SVM on Ngram Level TF IDF Vectors
# accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("SVM, N-Gram Vectors: ", accuracy)

In [62]:
# # RF on Count Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
# print ("RF, Count Vectors: ", accuracy)

# # RF on Word Level TF IDF Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("RF, WordLevel TF-IDF: ", accuracy)

In [61]:
# # Extereme Gradient Boosting on Count Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
# print ("Xgb, Count Vectors: ", accuracy)

# # Extereme Gradient Boosting on Word Level TF IDF Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
# print ("Xgb, WordLevel TF-IDF: ", accuracy)

# # Extereme Gradient Boosting on Character Level TF IDF Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
# print ("Xgb, CharLevel Vectors: ", accuracy)

In [139]:
from monkeylearn import MonkeyLearn

ml = MonkeyLearn('04836b26e06617e8ac7ff5062594f25454b78617')
data = ["Could reporters stop asking if political leaders 'believe' in climate change and start asking if they understand it instead",
       "NBA MVP Stephen Curry goes through his dribbling routine before Saturdays game, when the Golden State Warriors hosted the Denver Nuggets."]
model_id = 'cl_WDyr2Q4F'
result = ml.classifiers.classify(model_id, data)
print(result.body)

[{'text': "Could reporters stop asking if political leaders 'believe' in climate change and start asking if they understand it instead", 'external_id': None, 'error': False, 'classifications': [{'tag_name': 'Science & Technology', 'tag_id': 54974215, 'confidence': 0.323}]}, {'text': 'NBA MVP Stephen Curry goes through his dribbling routine before Saturdays game, when the Golden State Warriors hosted the Denver Nuggets.', 'external_id': None, 'error': False, 'classifications': [{'tag_name': 'Sports', 'tag_id': 54974218, 'confidence': 0.339}]}]


In [149]:
total_train_DF['full_text'].values.tolist()

['“Block Covid resources such as beds and release only on request\n\nDon’t help anyone who doesn’t tag “IYC”\n\nPrioritise help to journalists and influencers”\n\nInnocent Indians were losing their lives and Congress was still playing politics! https://t.co/iRmWv4Y7cU',
 'ToolKit by @INCIndia is another embarrassing testimony of how an electorally vanquished Congress,is driven not by Seva bhaav,during #COVID19 pandemic\n\nEven now,entire fulcrum of @RahulGandhi&amp;his politics is driven by visceral hatred for @narendramodi govt\n\n#CongressToolkitExposed https://t.co/pOiZxR9Dft',
 'Crenshaw claimed the GOP brief just asked the court a question about the constitutionality of states’ acts. \n\nNope. It called those acts unconstitutional, backed Texas’ overturn lawsuit, asked the court to halt the states’ certification of Biden’s wins: https://t.co/OHWSs77562',
 'RT @Ch_danishh: #کورونا_پھیلاتی_مریم\n\nTo all her supporters:\n\nSave yourselves &amp; your family as it is not suitable time

In [151]:
total_train_DF['full_text'].to_csv(r'test123.csv', index = False)

In [145]:
data2 = total_train_DF['full_text'].to_numpy()

In [150]:
result = ml.classifiers.classify(model_id, total_train_DF['full_text'].values.tolist())
print(result.body)

PlanQueryLimitError: Error PLAN_QUERY_LIMIT: Request Throttled. You do not have enough queries left to perform this action. 200 queries are required, but you have 94 queries remaining.

In [209]:
total_train_DF.to_csv(r'train_data.csv', index = False)